In [28]:
import ccxt, pandas as pd, indicateur_techniques as id, numpy as np, matplotlib.pyplot as plt

In [57]:
class Backtest:
	def __init__(self, capital):
		self.capital_initial = capital
		self.capital = capital

		self.positions_long = []
		self.quantite_position_long = []

		self.positions_short = []
		self.quantite_position_short = []


	def load_data(self, symbol='BTC/USDT', time='30m', length=500):
		#telecharcgement des donnée ohlcv
		self.exchange = ccxt.binance({
		'enableRateLimit': True,
		})
		try:
			ohlcv_data = self.exchange.fetch_ohlcv(symbol, time, limit=length)
			self.data = pd.DataFrame(ohlcv_data, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
			print("data dowload")

			self.data['position_long_open'] = None
			self.data['position_long_close'] = None

			self.data['position_short_open'] = None
			self.data['position_short_close'] = None

			self.data['evolution_price'] = None
		except ccxt.NetworkError:
			print("Problème lors du téléchargement des données")
		
	def open_long(self, close, pos, quantite=0.1):
		if not self.positions_long:
			self.positions_long.append(close)
			self.quantite_position_long.append(self.capital * quantite)
			print("open_long", close, "   ", pos)
			self.append_element_df("long_open", pos)
	
	def close_long(self, close, position, test=0):
		if self.positions_long:
			for pos in range(len(self.positions_long)):
				self.capital += (close - self.positions_long[pos]) * self.quantite_position_long[pos] # - (self.positions_long[pos] * self.quantite_position_long[pos] * 0.3
			self.positions_long.clear()
			self.quantite_position_long.clear()
			self.append_element_df("long_close", position)
			
			if test == 1:
				print("close garce au take_profit", close, "   ", position)
			elif test == 2:
				print("close garce au stop_loss", close, "   ", position)
			elif test == 0:
				print("close_short")
				print(close, "   ", position)
			else:
				print("choix_incorrect", test)
		
	def open_short(self, close, pos, quantite=0.1):
		if not self.positions_short:
			self.positions_short.append(close)
			self.quantite_position_short.append(self.capital * quantite)
			print("open_short", close, "   ", pos)
			self.append_element_df("short_open", pos)
	
	def close_short(self, close, position, test=0):
		if self.positions_short:
			for pos in range(len(self.positions_short)):
				self.capital += (self.positions_short[pos] - close)  * self.quantite_position_short[pos] # - (self.positions_short[pos] * self.quantite_position_short[pos] * 0.3)
			self.positions_short.clear()
			self.quantite_position_short.clear()
			self.append_element_df("short_close", position) 
			
			if test == 1:  	
				print("close garce au take_profit", close, "   ", position)
			elif test == 2:
				print("close garce au stop_loss", close, "   ", position)
			elif test == 0:
				print("close_short", close, "   ", position)
			else:
				print("choix_incorrect", test)
				
	
	def show_evolution_price(self):
		self.data.evolution_price.plot()
	
	def update(self, pos, close):
		"""
		close_actuel
		"""
		self.take_profit(close, pos)
		self.stop_loss(close, pos)
		self.data.at[pos, 'evolution_price'] = self.capital
		if self.positions_long:
			print("___  ", ((close / self.positions_long[0]) - 1) * 100, "  ___")
		if self.positions_short:
			print("***  ", ((self.positions_short[0] / close) - 1) * 100, "  ***")
		
	
	def take_profit(self, close, position):
		if self.positions_long:
			
			if ((close / self.positions_long[0]) - 1) * 100 > 2:
				print("take_profit_long")
				print(((close / self.positions_long[0]) - 1) * 100)
				self.close_long(close, position, 1)
				
	
		if self.positions_short:
			if ((close / self.positions_short[0]) - 1) * 100 < -2:
				print("take_profit_short")
				print(((close / self.positions_short[0]) - 1) * 100)
				self.close_short(close, position, 1)
	
	def stop_loss(self, close, position):
		
		if self.positions_long:
			if ((close / self.positions_long[0]) - 1) * 100 < -1:
				print("stop_loss_long")
				print(((close / self.positions_long[0]) - 1) * 100)
				self.close_long(close, position, 2)

		if self.positions_short:
			if ((close / self.positions_short[0]) - 1) * 100 > 1:
				print("stop_loss_short")
				print(((close / self.positions_short[0]) - 1) * 100)
				self.close_short(close, position, 2)

	def look_money(self):
		print(f"Votre porte feuille contient {self.capital}$")
	
	def rendement_final(self):
		self.rendement = ((self.capital - self.capital_initial) / self.capital_initial) * 100
		print(f"Le taux d'évolution est de {self.rendement}$")
	
	def append_element_df(self, style:str, position):
		"""
		style doit être égale a :
		-short_open
		-short_close
		-long_open
		-long_close
		et position a ou on est dans le df
		"""
		if style == "short_open":
			self.data.at[position, 'position_short_open'] = True
		elif style == "short_close":
			self.data.at[position, 'position_short_close'] = True
		elif style == "long_open":
			self.data.at[position, 'position_long_open'] = True
		elif style == "long_close":
			self.data.at[position, 'position_long_close'] = True
		else:
			print("Error, the style is not correct")
	
	def graphique(self, list_indicateur:list):
		#lsit_indicateur doit contenir les df de tout les indicateur utiliser
		
		positions_sc = np.where(self.data.position_short_close == True)[0]
		positions_lo = np.where(self.data.position_long_open == True)[0]
		positions_lc = np.where(self.data.position_long_close == True)[0]
		positions_so = np.where(self.data.position_short_open == True)[0]

		for indicateur in list_indicateur:
			indicateur.plot()
		
		move = self.data.close.mean() * 0.0007
		for pos in positions_so:
			x = pos
			y = self.data.close[pos]
			plt.plot(x, y+move,'o', marker='v', markersize=7, label='Triangles', color='green')
		for pos in positions_sc:
			x = pos
			y = self.data.close[pos]
			plt.plot(x, y+move,'o', marker='v', markersize=7, label='Triangles', color='red')
		for pos in positions_lo:
			x = pos
			y = self.data.close[pos]
			plt.plot(x, y-move,'o', marker='^', markersize=7, label='Triangles', color='green')
		for pos in positions_lc:
			x = pos
			y = self.data.close[pos]
			plt.plot(x, y-move,'o', marker='^', markersize=7, label='Triangles', color='red')
		self.data.close.plot()
	
	def trier_signal(self, series):
		result = []
		current_value = None
		for value in series:
			if value == current_value:
				result.append(None)
			else:
				current_value = value
				result.append(value)
		result[0] = None
		return result

In [58]:
backtest_test = Backtest(1000)
backtest_test.load_data(length=1000)
backtest_test.data["mm50"] = id.mm(backtest_test.data.close, 50)
backtest_test.data["mm100"] = id.mm(backtest_test.data.close, 100)
backtest_test.data["signal_long"] = backtest_test.data["mm100"] < backtest_test.data["mm50"]
backtest_test.data["signal_long"] = backtest_test.trier_signal(backtest_test.data["signal_long"])
backtest_test.data["signal_short"] = backtest_test.data["mm50"] < backtest_test.data["mm100"]
backtest_test.data["signal_short"] = backtest_test.trier_signal(backtest_test.data["signal_short"])
for i in range(len(backtest_test.data.close)):
	
	if backtest_test.data.signal_long[i] == True:
		backtest_test.open_long(backtest_test.data.close[i], i, 0.01)
	elif backtest_test.data.signal_long[i] == False:
		backtest_test.close_long(backtest_test.data.close[i], i)

	
	
	if backtest_test.data.signal_short[i] == True:
		backtest_test.open_short(backtest_test.data.close[i], i, 0.01)
	elif backtest_test.data.signal_short[i] == False:
		backtest_test.close_short(backtest_test.data.close[i], i)
	backtest_test.update(i, backtest_test.data.close[i])

data dowload
open_long 30756.65     99
___   0.0   ___
___   -0.08593263570642051   ___
___   -0.2492469108306694   ___
___   -0.19797344639290726   ___
___   -0.26995787902779345   ___
___   -0.37907249326568637   ___
___   -0.34269011742176225   ___
___   -0.14926853217108738   ___
___   -0.29086392698816077   ___
___   -0.4294030721811337   ___
___   -0.3788774135024542   ___
___   -0.304291917357713   ___
___   -0.47482414372176374   ___
___   -0.4638671636865599   ___
___   -0.3789749533840703   ___
___   -0.3397314076793223   ___
___   -0.4273222213732608   ___
___   -0.5223260660702667   ___
___   -0.3368052112307396   ___
___   -0.41210599984068397   ___
___   -0.33426917430865455   ___
___   -0.28169517811595934   ___
___   -0.06626209291323848   ___
___   0.5996426789003362   ___
___   1.0143822555447324   ___
___   0.7985915241094155   ___
___   0.9284821331321913   ___
___   0.7978762316442012   ___
___   0.7943973091997902   ___
___   0.7911784931063703   ___
___   1.43868

In [59]:
%matplotlib
backtest_test.graphique([backtest_test.data.mm50, backtest_test.data.mm100])

Using matplotlib backend: TkAgg


C:\Users\JB\AppData\Local\Temp\ipykernel_4252\2037157073.py:170: UserWarning: marker is redundantly defined by the 'marker' keyword argument and the fmt string "o" (-> marker='o'). The keyword argument will take precedence.
  plt.plot(x, y+move,'o', marker='v', markersize=7, label='Triangles', color='green')
C:\Users\JB\AppData\Local\Temp\ipykernel_4252\2037157073.py:174: UserWarning: marker is redundantly defined by the 'marker' keyword argument and the fmt string "o" (-> marker='o'). The keyword argument will take precedence.
  plt.plot(x, y+move,'o', marker='v', markersize=7, label='Triangles', color='red')
C:\Users\JB\AppData\Local\Temp\ipykernel_4252\2037157073.py:178: UserWarning: marker is redundantly defined by the 'marker' keyword argument and the fmt string "o" (-> marker='o'). The keyword argument will take precedence.
  plt.plot(x, y-move,'o', marker='^', markersize=7, label='Triangles', color='green')
C:\Users\JB\AppData\Local\Temp\ipykernel_4252\2037157073.py:182: UserWar